import pandas as pd
import numpy as np

In [2]:
# Все операции текста сводятся лишь к lowercase, так как нужны рекомендации, написанные грамотно -> нельзя приводить текст к нормальной форме, удалять цифры и т.д.
query_popularity = pd.read_csv('query_popularity.csv')
query_popularity['query'] = query_popularity['query'].str.lower()
search_hisory = pd.read_csv('search_history.csv')
# Переведем весь текст в нижний регистр, чтобы экономить память
search_hisory.UQ = search_hisory.UQ.str.lower()

### Выделим выборку для моделирования рекомендаций
Необходимо, чтобы было минимум 2 запроса
### Уберем ботов
Юзер будет являться ботом, если у него будет больше 30 уникальных запросов.

In [3]:
searchers_zaps = search_hisory[['wbuser_id', 'UQ', 'weekday', 'time']].drop_duplicates(['wbuser_id', 'UQ'])

In [4]:
searchers = searchers_zaps.groupby('wbuser_id', as_index=False).agg({'UQ':'count'}).rename(columns={"UQ":'how_many_requests'})

In [5]:
# Кол-во предположительных ботов
bots_num = searchers[~(searchers.how_many_requests<=30 )& (searchers.how_many_requests>=2)].shape[0]
bots_num

44324

In [8]:
# Кол-во клиентов, на которых можно учить алгоритм делать рекомендации
real_num = searchers[(searchers.how_many_requests>=2)].shape[0]
real_num

4893526

In [9]:
# посомотрим, какой процент ботов мы отсеиваем (1%)
bots_num/real_num

0.009057681516354466

In [10]:
# Их доля запросов приходящщаяся в выборке на ботов (31%). Достаточно большое число, которое нужно отсеивать
search_hisory[search_hisory.wbuser_id.isin(searchers[~(searchers.how_many_requests<=30 )& (searchers.how_many_requests>=2)].wbuser_id)].shape[0]/\
search_hisory[search_hisory.wbuser_id.isin(searchers[ (searchers.how_many_requests>=2)].wbuser_id)].shape[0]

0.3139739510279077

In [11]:
# оставим только запросы, находящиеся в популярных, для того, чтобы минимизировать шанс предложить текст с опечатками и продвигающими запросами
searchers_zaps = searchers_zaps[searchers_zaps.drop_duplicates(['wbuser_id', 'UQ', 'weekday']).UQ.isin(query_popularity['query'])]

In [12]:
### Клиенты, которые подходят для модели
good_id = searchers[(searchers.how_many_requests<=30 )& (searchers.how_many_requests>=2)].wbuser_id

In [13]:
searchers_zaps = searchers_zaps[searchers_zaps.wbuser_id.isin(good_id)]

In [14]:
# группируем запросы клиентов по дню. Считаем, что серией является поиск за один день. Так как все запросы отсортированы по времени, не страшно, если сессия запросов будет не совсем точно определена
grouped_zaps = searchers_zaps.groupby(['wbuser_id', 'weekday'])['UQ'].agg(lambda x: x.tolist())
good_series = np.array(grouped_zaps[grouped_zaps.str.len()>=2])

In [15]:
# Кол-во хороших серий
good_series.shape

(1298963,)

### Обучим w2v 

In [16]:
from gensim.models import Word2Vec

In [17]:
model = Word2Vec(sentences=good_series, vector_size=100, window=3, min_count=1, workers=4)

In [ ]:
model.save("word2vec.model")